In [1]:
import pandas as pd

In [4]:
# Load the data set (all transactions in 1 year)
data = pd.read_csv('072016_072017.csv')

In [6]:
# View the original data
data.head(10)

,Date,Amount,Account,Balance,Category,Description,Transaction Type,Agent,Unnamed: 8
0,"Jul 7, 2559 BE, 5:39:02 PM",A$0.00,Wallet,A$0.00,NaN,,Opening Balance,NaN,NaN
1,"Jul 7, 2559 BE, 5:41:22 PM",A$12.00,Wallet,(A$12.00),Groceries,Woolworth,Expense,NaN,NaN
2,"Jul 7, 2559 BE, 5:41:49 PM",A$6.00,Wallet,(A$18.00),Home: Other,Clothes hanger,Expense,NaN,NaN
3,"Jul 7, 2559 BE, 5:42:30 PM",A$458.00,Wallet,A$440.00,NaN,,Balance Adjustment,NaN,NaN
4,"Jul 7, 2559 BE, 11:52:46 PM",A$20.00,Wallet,A$420.00,Dining,,Expense,NaN,NaN
5,"Jul 8, 2559 BE, 7:01:39 PM",A$3.30,Wallet,A$416.70,Groceries,,Expense,NaN,NaN
6,"Jul 8, 2559 BE, 7:01:56 PM","A$1,200.00",Wallet,(A$783.30),Home: Rent,550 Bond + 650 July,Expense,NaN,NaN
7,"Jul 9, 2559 BE, 12:07:44 PM",A$20.00,Wallet,(A$803.30),Transportation: Myki,,Expense,NaN,NaN
8,"Jul 9, 2559 BE, 7:18:26 PM",A$12.00,Wallet,(A$815.30),Dining,,Expense,NaN,NaN
9,"Jul 9, 2559 BE, 8:56:00 PM",A$5.00,Wallet,(A$820.30),Study Goods,,Expense,NaN,NaN


In [12]:
# Filter data
expenses = data[data['Transaction Type'] == 'Expense']
expenses = expenses[['Date', 'Amount', 'Category', 'Description']]

In [13]:
len(expenses)

466

In [14]:
# Filtered Data
expenses.head(10)

,Date,Amount,Category,Description
1,"Jul 7, 2559 BE, 5:41:22 PM",A$12.00,Groceries,Woolworth
2,"Jul 7, 2559 BE, 5:41:49 PM",A$6.00,Home: Other,Clothes hanger
4,"Jul 7, 2559 BE, 11:52:46 PM",A$20.00,Dining,
5,"Jul 8, 2559 BE, 7:01:39 PM",A$3.30,Groceries,
6,"Jul 8, 2559 BE, 7:01:56 PM","A$1,200.00",Home: Rent,550 Bond + 650 July
7,"Jul 9, 2559 BE, 12:07:44 PM",A$20.00,Transportation: Myki,
8,"Jul 9, 2559 BE, 7:18:26 PM",A$12.00,Dining,
9,"Jul 9, 2559 BE, 8:56:00 PM",A$5.00,Study Goods,
10,"Jul 11, 2559 BE, 7:01:57 PM",A$5.50,Groceries,Rice and milk
11,"Jul 11, 2559 BE, 7:02:18 PM",A$20.00,Home: Other,Rice cooker


In [18]:
expenses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466 entries, 1 to 467
Data columns (total 4 columns):
Date           466 non-null object
Amount         466 non-null object
Category       466 non-null object
Description    466 non-null object
dtypes: object(4)
memory usage: 18.2+ KB


In [19]:
# See unique category
expenses['Category'].value_counts()

Dining                     194
Groceries                  140
Entertainment               36
Clothing                    25
Study Goods                 21
Transportation: Myki        20
Home: Rent                  12
Utilities: Cell Phone        8
Transportation: Other        6
Home: Other                  2
Utilities: Cable             1
Utilities: Electricity       1
Name: Category, dtype: int64

In [28]:
# Combine categories with the same 'CatName:' together
for catName in expenses['Category'].unique():
    if(':' in catName):
        head, sep, tail = catName.partition(':')
        expenses.loc[expenses['Category'] == catName, 'Category'] = head

In [29]:
# Check the result
expenses['Category'].value_counts()

Dining            194
Groceries         140
Entertainment      36
Transportation     26
Clothing           25
Study Goods        21
Home               14
Utilities          10
Name: Category, dtype: int64

In [ ]:
# Change 'Date' to datetime type


In [ ]:
# Change 'Amount' to number type